In [1]:
from pyspark.sql import SparkSession

In [2]:
spark=SparkSession.builder.appName('Join').getOrCreate()

In [3]:
spark

In [23]:
customer=spark.read.csv(r'C:\Users\pavan\OneDrive\Documents\My Tableau Repository\tableau_tutorial\EU-Training-Datasets\EU-Training-Datasets\EU Training Datasets\TableauProject-SalesDashboard\Customers.csv',sep=';',inferSchema=True,header=True)

In [24]:
customer.show()

+-----------+--------------------+
|Customer ID|       Customer Name|
+-----------+--------------------+
|   AA-10315|          Alex Avila|
|   AA-10375|        Allen Armold|
|   AA-10480|        Andrew Allen|
|   AA-10645|       Anna Andreadi|
|   AB-10015|       Aaron Bergman|
|   AB-10060|     Adam Bellavance|
|   AB-10105|       Adrian Barton|
|   AB-10150|         Aimee Bixby|
|   AB-10165|         Alan Barnes|
|   AB-10255|Alejandro Ballentine|
|   AB-10600|           Ann Blume|
|   AC-10420|       Alyssa Crouse|
|   AC-10450|             Amy Cox|
|   AC-10615|           Ann Chong|
|   AC-10660|          Anna Chung|
|   AD-10180|      Alan Dominguez|
|   AF-10870|        Art Ferguson|
|   AF-10885|          Art Foster|
|   AG-10270|     Alejandro Grove|
|   AG-10300| Aleksandra Gannaway|
+-----------+--------------------+
only showing top 20 rows



In [25]:
orders=spark.read.csv(r"C:\Users\pavan\OneDrive\Documents\My Tableau Repository\tableau_tutorial\EU-Training-Datasets\EU-Training-Datasets\EU Training Datasets\TableauProject-SalesDashboard\Orders.csv",sep=';',inferSchema=True,header=True)

In [32]:
orders.printSchema()

root
 |-- Row ID: integer (nullable = true)
 |-- Order ID: string (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Ship Date: string (nullable = true)
 |-- Ship Mode: string (nullable = true)
 |-- Customer ID: string (nullable = true)
 |-- Segment: string (nullable = true)
 |-- Postal Code: integer (nullable = true)
 |-- Product ID: string (nullable = true)
 |-- Sales: integer (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Discount: double (nullable = true)
 |-- Profit: integer (nullable = true)



In [45]:
orders.groupBy('Segment','Ship Mode').min('Sales').show()

+-----------+--------------+----------+
|    Segment|     Ship Mode|min(Sales)|
+-----------+--------------+----------+
|  Corporate|  Second Class|        26|
|   Consumer|      Same Day|        29|
|  Corporate|      Same Day|       158|
|   Consumer|  Second Class|         6|
|Home Office|   First Class|        30|
|Home Office|Standard Class|         3|
|  Corporate|   First Class|        15|
|Home Office|      Same Day|        78|
|   Consumer|Standard Class|         3|
|   Consumer|   First Class|         6|
|Home Office|  Second Class|         5|
|  Corporate|Standard Class|         8|
+-----------+--------------+----------+



In [79]:
df=orders.join(customer,'Customer ID',how='outer')
#sort('Sales').show()

In [91]:
df.sort('Quantity',ascending=False).show()

+-----------+------+--------------+----------+----------+--------------+-----------+-----------+---------------+-------+--------+--------+-------+------------------+
|Customer ID|Row ID|      Order ID|Order Date| Ship Date|     Ship Mode|    Segment|Postal Code|     Product ID|  Sales|Quantity|Discount| Profit|     Customer Name|
+-----------+------+--------------+----------+----------+--------------+-----------+-----------+---------------+-------+--------+--------+-------+------------------+
|   DK-12835|  1430|US-2021-164448|31/10/2021| 4/11/2021|  Second Class|  Corporate|      93905|OFF-LA-10003510|  42742|      14|     0.0|1966132|   Damala Kotsonis|
|   JM-15250|  8075|CA-2023-151750|  1/1/2023|  5/1/2023|Standard Class|   Consumer|      77340|OFF-BI-10000343|  13748|      14|     0.8|-226842|      Janet Martin|
|   DL-13315|  9516|CA-2020-154165|17/02/2020|24/02/2020|Standard Class|   Consumer|      60653|OFF-AR-10003631|  54208|      14|     0.2|  88088| Delfina Latchford|
|   

In [101]:
from pyspark.sql.functions import lit

In [88]:
from pyspark.sql.functions import max

In [98]:
max_q=df.agg(max('Quantity')).collect()[0][0]

In [107]:
df1=df

In [109]:
df1=df1.withColumn('Max',lit(max_q))

In [131]:
df.filter(df.Sales >= 23962656).show()

+-----------+------+--------------+----------+---------+------------+--------+-----------+---------------+--------+--------+--------+--------+-------------+
|Customer ID|Row ID|      Order ID|Order Date|Ship Date|   Ship Mode| Segment|Postal Code|     Product ID|   Sales|Quantity|Discount|  Profit|Customer Name|
+-----------+------+--------------+----------+---------+------------+--------+-----------+---------------+--------+--------+--------+--------+-------------+
|   AG-10675|   400|CA-2022-108987|  8/9/2022|10/9/2022|Second Class|Consumer|      77036|FUR-BO-10004834|23962656|       4|    0.32|-3171528|  Anna Gayman|
+-----------+------+--------------+----------+---------+------------+--------+-----------+---------------+--------+--------+--------+--------+-------------+



In [111]:
df.show()

+-----------+------+--------------+----------+----------+--------------+--------+-----------+---------------+-------+--------+--------+--------+-------------+
|Customer ID|Row ID|      Order ID|Order Date| Ship Date|     Ship Mode| Segment|Postal Code|     Product ID|  Sales|Quantity|Discount|  Profit|Customer Name|
+-----------+------+--------------+----------+----------+--------------+--------+-----------+---------------+-------+--------+--------+--------+-------------+
|   AA-10315|  1160|CA-2023-147039|29/06/2023|  4/7/2023|Standard Class|Consumer|      55407|OFF-AP-10000576|  36294|       3|     0.0|   90735|   Alex Avila|
|   AA-10315|  1161|CA-2023-147039|29/06/2023|  4/7/2023|Standard Class|Consumer|      55407|OFF-BI-10004654|   1154|       2|     0.0|     577|   Alex Avila|
|   AA-10315|  1300|CA-2021-121391| 4/10/2021| 7/10/2021|   First Class|Consumer|      94109|OFF-ST-10001590|   2696|       2|     0.0|   70096|   Alex Avila|
|   AA-10315|  2230|CA-2020-128055|31/03/2020|

In [113]:
count_customers=df.groupBy('Customer ID').count().sort('count')

In [136]:
count_customers=count_customers.withColumnRenamed('count','count_orders')

In [137]:
count_customers.printSchema()

root
 |-- Customer ID: string (nullable = true)
 |-- count_orders: long (nullable = false)



In [ ]:
df.filer

In [140]:
count_customers.filter(count_customers.count_orders > 1).show()

+-----------+------------+
|Customer ID|count_orders|
+-----------+------------+
|   PB-19210|           2|
|   PH-18790|           2|
|   BO-11425|           2|
|   AR-10570|           2|
|   SH-20635|           2|
|   NB-18580|           2|
|   EL-13735|           2|
|   TS-21085|           2|
|   RM-19750|           2|
|   MG-18205|           2|
|   LH-16750|           3|
|   JR-15670|           3|
|   MW-18235|           3|
|   PC-19000|           3|
|   PK-18910|           3|
|   RS-19870|           3|
|   FW-14395|           3|
|   ML-18265|           3|
|   LT-16765|           3|
|   RH-19555|           3|
+-----------+------------+
only showing top 20 rows



In [158]:
count_customers.agg( {'count_orders' : 'max','count_orders' : 'min'} ).show()

+-----------------+
|min(count_orders)|
+-----------------+
|                1|
+-----------------+



In [159]:
from pyspark.sql import functions as F

In [168]:
count_customers.agg( F.min(count_customers.count_orders),F.max(count_customers.count_orders) ).show()

+-----------------+-----------------+
|min(count_orders)|max(count_orders)|
+-----------------+-----------------+
|                1|               37|
+-----------------+-----------------+



In [170]:
m_c=count_customers.agg( F.min(count_customers.count_orders),F.max(count_customers.count_orders) )

In [172]:
m_c.collect()[0][1]

37